<h3>This file merges the GAN and sensor data. Not only it is needed for algorithm but also for comparison between the two files </h3>

In [1]:
#packages 

import pandas as pd
import numpy as np
import boto3

In [2]:
#file import from S3

s3 = boto3.client('s3')

bucket = 'flood-prediction-master-dataset'
key = 'ml-sensor-data-15-min/ml_sensor_data_15_min.csv'

obj = s3.get_object(Bucket= bucket,Key= key)
sensor_file = pd.read_csv(obj['Body'],names=["time","river","rain","temperature","wind_direction","wind_speed"])

key = 'ml-sensor-data-15-min/gan_data_15_min.csv'

obj = s3.get_object(Bucket= bucket,Key= key)
gan_file = pd.read_csv(obj['Body'])  #GAN was saved with header. hence no need for stating column names

In [3]:
gan_file

,time,river,rain,temperature,wind_direction,wind_speed
0,2020-07-11T14:45,7.595,0.0,18.37,270,6.433
1,2020-07-11T14:30,7.500,0.0,17.46,267,6.048
2,2020-07-11T14:15,7.332,0.0,16.35,268,6.334
3,2020-07-11T14:00,7.202,0.0,16.41,267,6.108
4,2020-07-11T13:45,7.109,0.1,16.83,268,5.325
...,...,...,...,...,...,...
2753,2020-06-12T01:00,6.078,0.0,11.94,4,4.914
2754,2020-06-12T00:45,5.702,0.1,11.84,5,5.201
2755,2020-06-12T00:30,5.332,0.0,11.71,8,5.099
2756,2020-06-12T00:15,4.932,0.0,11.61,4,5.554


In [4]:
sensor_file

,time,river,rain,temperature,wind_direction,wind_speed
0,2020-07-11T14:45,7.659,0.0,18.37,270.0,6.433
1,2020-07-11T14:30,7.578,0.0,17.46,267.0,6.048
2,2020-07-11T14:15,7.442,0.0,16.35,268.0,6.334
3,2020-07-11T14:00,7.252,0.0,16.41,267.0,6.108
4,2020-07-11T13:45,7.019,0.0,16.83,268.0,5.325
...,...,...,...,...,...,...
2753,2020-06-12T01:00,6.078,0.0,11.94,4.0,4.914
2754,2020-06-12T00:45,5.702,0.0,11.84,5.0,5.201
2755,2020-06-12T00:30,5.332,0.0,11.71,8.0,5.099
2756,2020-06-12T00:15,4.932,0.0,11.61,4.0,5.554


In [5]:
#converting the string datatype of time values to datetime

sensor_file['timerecorded'] = pd.to_datetime(sensor_file['time'])
gan_file['timerecorded'] = pd.to_datetime(gan_file['time'])

In [6]:
#dropping the redundant column

sensor_file.drop(['time'],axis=1,inplace=True)
gan_file.drop(['time'],axis=1,inplace=True)

In [7]:
#sorting by datetime column

gan_file.sort_values(by=['timerecorded'],inplace=True)
sensor_file.sort_values(by=['timerecorded'],inplace=True)

In [8]:
#quicksight imports all data from all the csv files from the folder and files specified.
#Hence, a distinguishing column is required. 

gan_file['source'] = 'GAN'
sensor_file['source'] = 'SENSOR'  

In [9]:
#merging both datasets

final_file_15_min = gan_file.append(sensor_file) 

In [10]:
print("Mean of River GAN: "+str(round(gan_file['river'].mean(),4))+" Mean of River Sensor: "+str(round(sensor_file['river'].mean(),4)))

print("Mean of Rainfall GAN: "+str(round(gan_file['rain'].mean(),4))+" Mean of Rainfall Sensor: "+str(round(sensor_file['rain'].mean(),4)))

print("Mean of Temperature GAN: "+str(round(gan_file['temperature'].mean(),4))+" Mean of Temperature Sensor: "+str(round(sensor_file['temperature'].mean(),4)))

print("Mean of wind direction GAN: "+str(round(gan_file['wind_direction'].mean(),4))+" Mean of wind direction Sensor: "+str(round(sensor_file['wind_direction'].mean(),4)))

print("Mean of wind speed GAN: "+str(round(gan_file['wind_speed'].mean(),4))+" Mean of wind speed Sensor: "+str(round(sensor_file['wind_speed'].mean(),4)))

Mean of River GAN: 5.3616 Mean of River Sensor: 5.3838
Mean of Rainfall GAN: 0.0521 Mean of Rainfall Sensor: 0.037
Mean of Temperature GAN: 15.6732 Mean of Temperature Sensor: 15.6295
Mean of wind direction GAN: 182.7169 Mean of wind direction Sensor: 182.7676
Mean of wind speed GAN: 5.4103 Mean of wind speed Sensor: 5.4429


In [11]:
final_file_15_min

,river,rain,temperature,wind_direction,wind_speed,timerecorded,source
2757,4.540,0.0,11.53,7.0,5.272,2020-06-12 00:00:00,GAN
2756,4.932,0.0,11.61,4.0,5.554,2020-06-12 00:15:00,GAN
2755,5.332,0.0,11.71,8.0,5.099,2020-06-12 00:30:00,GAN
2754,5.702,0.1,11.84,5.0,5.201,2020-06-12 00:45:00,GAN
2753,6.078,0.0,11.94,4.0,4.914,2020-06-12 01:00:00,GAN
...,...,...,...,...,...,...,...
4,7.019,0.0,16.83,268.0,5.325,2020-07-11 13:45:00,SENSOR
3,7.252,0.0,16.41,267.0,6.108,2020-07-11 14:00:00,SENSOR
2,7.442,0.0,16.35,268.0,6.334,2020-07-11 14:15:00,SENSOR
1,7.578,0.0,17.46,267.0,6.048,2020-07-11 14:30:00,SENSOR


In [12]:
#reset file index to current dataframe 

final_file_15_min.reset_index(drop=True, inplace=True)

In [13]:
#save as a csv file locally

final_file_15_min.to_csv("final_data_15_min.csv")

In [14]:
import datetime
import tarfile

import boto3 # AWS SDK for python. Provides low-level access to AWS services
from sagemaker import get_execution_role
import sagemaker

m_boto3 = boto3.client('sagemaker') 

sess = sagemaker.Session()

region = sess.boto_session.region_name

bucket = 'flood-prediction-master-dataset'  #  Bucket name where the file is to be saved

print('Using bucket ' + bucket)

# save data to S3. key_prefix is the directory path. path is the local file name which will be saved in S3. It has to be 
# the same file name as saved locally and bucket is the bucket name 

upload = sess.upload_data(path='final_data_15_min.csv', bucket=bucket, key_prefix='final-dataset-15-min')

Using bucket flood-prediction-master-dataset
